In [6]:
import requests
import csv

# Define the API key and base URLs
API_KEY = 'API Key'  # API key removed for privacy purpose
endpoint_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
details_url = 'https://maps.googleapis.com/maps/api/place/details/json'

# Read coordinates from CSV file
input_csv = 'county_pts.csv'  # CSV file with Lat_Y and Long_X columns

# Initialize a counter for output files
file_counter = 1

# Open input CSV and iterate through each row for Lat_Y and Long_X
with open(input_csv, mode='r', newline='', encoding='utf-8') as input_file:
    csv_reader = csv.DictReader(input_file)
    
    for row in csv_reader:
        latitude = row['Lat_Y']
        longitude = row['Long_X']
        location = f"{latitude},{longitude}"

        # Define the output file name based on the counter
        output_csv = f'Battlefield{file_counter}.csv'
        
        # Define parameters for the API request using coordinates from the CSV file
        params = {
            'location': location,  # Coordinates from the CSV
            'radius': '50000',  # 50km radius
            'keyword': 'Battlefield',  # Search keyword
            'key': API_KEY
        }

        # Send the request to the Places API
        response = requests.get(endpoint_url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            results = response.json().get('results', [])
            if results:
                # Open CSV file for output
                with open(output_csv, mode='w', newline='', encoding='utf-8') as output_file:
                    writer = csv.writer(output_file)
                    # Write header row to the CSV file, including City, Type, and Description
                    writer.writerow(['Place Name', 'Address', 'City', 'Rating', 'Total Ratings', 'Latitude', 'Longitude', 'Type', 'Description'])

                    # Iterate through each place result
                    for place in results:
                        name = place.get('name', 'No name available')
                        formatted_address = place.get('vicinity', 'No address available')
                        rating = place.get('rating', 'No rating available')
                        total_ratings = place.get('user_ratings_total', 'No ratings available')
                        lat = place['geometry']['location'].get('lat', 'No latitude available')
                        lng = place['geometry']['location'].get('lng', 'No longitude available')
                        types = ', '.join(place.get('types', ['No types available']))

                        # Split the formatted_address to extract City
                        formatted_address_parts = formatted_address.split(', ')
                        city = formatted_address_parts[-1] if len(formatted_address_parts) >= 1 else 'No city available'

                        # Get place details if place_id is available
                        place_id = place.get('place_id', None)
                        description = 'No description available'
                        
                        if place_id:
                            details_params = {
                                'place_id': place_id,
                                'fields': 'name,formatted_address,geometry,place_id,rating,user_ratings_total,reviews,types',
                                'key': API_KEY
                            }
                            details_response = requests.get(details_url, params=details_params)

                            # Extract description from the reviews if available
                            if details_response.status_code == 200:
                                details_result = details_response.json().get('result', {})
                                if 'reviews' in details_result and details_result['reviews']:
                                    description = details_result['reviews'][0].get('text', 'No description available')

                        # Print place details for debugging
                        print(f"Place Name: {name}")
                        print(f"Address: {formatted_address}")
                        print(f"City: {city}")
                        print(f"Rating: {rating}")
                        print(f"Total Ratings: {total_ratings}")
                        print(f"Latitude: {lat}")
                        print(f"Longitude: {lng}")
                        print(f"Type: {types}")
                        print(f"Description: {description}\n")

                        # Write place details to the CSV file
                        writer.writerow([name, formatted_address, city, rating, total_ratings, lat, lng, types, description])

                # Increment the file counter for the next output file
                file_counter += 1
            else:
                print(f"No results found for location: {location}")
        else:
            print(f"Error: Unable to fetch places data for location {location}. Status Code: {response.status_code}")
            print(f"Response: {response.text}")


No results found for location: 37.87830559000,-96.23261109000
No results found for location: 39.78823052000,-95.14682688000
No results found for location: 39.82802392000,-97.65063122000
No results found for location: 39.78510569000,-100.45990796000
No results found for location: 39.78493260000,-99.34703017000
No results found for location: 38.45674951000,-96.15264367000
No results found for location: 37.99963107000,-101.79127051000
No results found for location: 38.91700124000,-101.76358082000
No results found for location: 39.29703883000,-96.73535445000
No results found for location: 38.91509268000,-99.31723721000
No results found for location: 37.64795292000,-98.73960439000
Place Name: Battle of Punished Woman's Fork National Historic Site
Address: 67871, Scott City
City: Scott City
Rating: 4.8
Total Ratings: 73
Latitude: 38.6435538
Longitude: -100.9279405
Type: tourist_attraction, point_of_interest, establishment
Description: This was maybe the highlight of our trip. We came to the 

In [1]:
import pandas as pd
import os
import geojson
import json


In [2]:
# Specify the folder containing Battlefield CSV files
folder_path = 'D:\_Bootcamp\_Project3\Battlefield'

# Create an empty list to hold DataFrames
df_list = []

# Loop through all files in the folder
for file in os.listdir(folder_path):
    if file.endswith('.csv'):  # Ensure only CSV files are processed
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)  # Read each CSV file into a DataFrame
        df_list.append(df)  # Append the DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(df_list, ignore_index=True)

# Show the merged DataFrame
print(merged_df)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('Battlefield/merged_battlefields.csv', index=False)

                                           Place Name  \
0   Battle of Punished Woman's Fork National Histo...   
1                      Nerf Warz Battlefield & Arcade   
2   Battle of Punished Woman's Fork National Histo...   
3                              Black Jack Battlefield   
4                                          Fort Titus   
..                                                ...   
67                Battle of Carthage Civil War Museum   
68                           Battle Of Beecher Island   
69                                     Beecher Island   
70                                     Armel Cemetery   
71                                         Fort Titus   

                          Address        City               Rating  \
0               67871, Scott City  Scott City                  4.8   
1      6821 E Kellogg Dr, Wichita     Wichita                  3.8   
2               67871, Scott City  Scott City                  4.8   
3       163 E 2000 Rd, Wellsville  

In [4]:
# Specify the folder containing Historical CSV files
folder_path = 'D:\_Bootcamp\_Project3\Historical'

# Create an empty list to hold DataFrames
df_list = []

# Loop through all files in the folder
for file in os.listdir(folder_path):
    if file.endswith('.csv'):  # Ensure only CSV files are processed
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)  # Read each CSV file into a DataFrame
        df_list.append(df)  # Append the DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
merged_hist_df = pd.concat(df_list, ignore_index=True)

# Show the merged DataFrame
print(merged_hist_df)

# Optionally, save the merged DataFrame to a new CSV file
merged_hist_df.to_csv('Historical/merged_historical.csv', index=False)

                                            Place Name  \
0                  Greenwood County Historical Society   
1                                           Teter Rock   
2                      Historic Frisco Railroad Bridge   
3                      Coffey County Historical Museum   
4                                   Hartford Calaboose   
...                                                ...   
2040                           Fromm Stone Arch Bridge   
2041                    Kansas' Oldest Swinging Bridge   
2042  Caney Valley Historical Society & Museum Complex   
2043                        South School historic site   
2044                    Riverside Park Dinosaur Statue   

                          Address          City  Rating  Total Ratings  \
0            120 W 4th St, Eureka        Eureka     5.0              2   
1              2555 300th, Eureka        Eureka     4.8            173   
2               M98V+33, Keighley      Keighley     5.0              2   
3      

In [2]:
# Load the haunted place csv file
haunted = "Data/csv/Haunted_Place_KS.csv"  

# Load csv file with ISO cp endcoding because utf-8 doesn't work
try:
    data = pd.read_csv(haunted, encoding='ISO-8859-1')  
except UnicodeDecodeError:
    data = pd.read_csv(haunted, encoding='cp1252')  

# Function to convert csv rows to GeoJSON features
def csv_to_geojson(data, lat_column, lon_column):
    features = []
    for _, row in data.iterrows():
        point = geojson.Point((float(row[lon_column]), float(row[lat_column])))
        properties = row.drop([lat_column, lon_column]).to_dict() 
        feature = geojson.Feature(geometry=point, properties=properties)
        features.append(feature)
    return geojson.FeatureCollection(features)

# Convert to GeoJSON
lat_column = 'latitude'  
lon_column = 'longitude' 
geojson_data = csv_to_geojson(data, lat_column, lon_column)

# Save the GeoJSON to a file
output_file = "Data/geojson/Haunted_Place_KS.geojson"
with open(output_file, "w") as f:
    geojson.dump(geojson_data, f)

print(f"GeoJSON file created: {output_file}")


GeoJSON file created: Data/geojson/Haunted_Place_KS.geojson


In [3]:
# Load and read the GeoJSON file for verification
geojson_file_path = 'Data/geojson/Haunted_Place_KS.geojson' 

with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

# Display the GeoJSON content using pretty print
print(json.dumps(geojson_data, indent=2))  


{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          -96.299018,
          39.016088
        ]
      },
      "properties": {
        "city": "Alma",
        "description": "The Alma Cemetery is haunted for two reasons. One is the people who are buried there and refuse to leave. Some pictures have been taken during the night up there. (It's the best time.) The pictures were unbeleivable, there were orbs and mist figures in t",
        "location": "Alma Cemetery",
        "state": "Kansas"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          -96.289219,
          39.018726
        ]
      },
      "properties": {
        "city": "Alma",
        "description": "Wabaunsee High School is the Haunted by past students and teachers. Most of the students groups are those of students who never got to graduate from the

In [4]:
# Load the CSV file, use encodling latin1 because is only one works
historical = "Data/csv/KS_Historical.csv"  
data_df = pd.read_csv(historical, encoding='latin1')

# Create GeoJSON structure
geojson = {
    "type": "FeatureCollection",
    "features": []
}

# Iterate over DataFrame rows and convert each row to a GeoJSON feature
for _, row in data_df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["Longitude"], row["Latitude"]]  # Adjust these column names if necessary
        },
        "properties": {
            "Place_Name": row["Place_Name"],
            "City": row["City"],
            "State": row["State"],
            "Rating": row["Rating"],
            "Total_Ratings": row["Total_Rati"],
            "Type": row["Type"],
        }
    }
    geojson["features"].append(feature)

# Save to GeoJSON file
output_path = "Data/geojson/KS_historical.geojson"
with open(output_path, 'w') as geojson_file:
    json.dump(geojson, geojson_file)

print(f"GeoJSON file created at {output_path}")


GeoJSON file created at Data/geojson/KS_historical.geojson


In [5]:
# Load and read the GeoJSON file for verification
geojson_file_path = 'Data/geojson/KS_historical.geojson' 

with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

# Display the GeoJSON content using pretty print
print(json.dumps(geojson_data, indent=2))  


{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          -96.2951936,
          37.8223962
        ]
      },
      "properties": {
        "Place_Name": "Greenwood County Historical Society",
        "City": "Eureka",
        "State": "Kansas",
        "Rating": 5.0,
        "Total_Ratings": 2,
        "Type": "museum, tourist_attraction, point_of_interest, establishment"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          -96.4235637,
          38.0295627
        ]
      },
      "properties": {
        "Place_Name": "Teter Rock",
        "City": "Eureka",
        "State": "Kansas",
        "Rating": 4.8,
        "Total_Ratings": 173,
        "Type": "point_of_interest, establishment"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          -